In [1]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.compose import make_column_transformer
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.utils import resample
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE

- ohe with logreg - 0.712
- ohe with logreg with age smoothing - 0.713
- ohe with logreg with age smoothing with license category?
- ohe with lighGBM - 0.716
- mean encoding with logreg - 0.6915
- mean encoding with sampling with logreg - 0.7
- mean encoding with lightGBM - 0.716
- label encoding with lightGBM with over-sampling with q1+q2 - 0.683
- label encoding with lightGBM with q1+q2 - 0.709
- label encoding with lightGBM with q1 - 0.713
- label encoding with lightGBM with over-sampling with q1 - 0.698
### to do:
- try qiwi
- more new features: registration, registration_type, birthplace, birthplace_type
- smoothing
- normalization
- sampling
- check precision/recall
- check what category of users model predict better and worse
### probability of road accident depends on:
- опыт вождения
  - как давно человек водит машину
    - реальный стаж в годах
  - есть ли своя машина
    - зарплата
    - наличие семьи
    - где живет: за городом/в городе; в Москве/в других городах
  - как регулярно
  - общий пробег в км
- стиль вождения
  - куда и где ездит: на работу/в магазин/в бар, в аварийно-опасном районе
  - "уровень культуры"
  - образование
    - зарплата
    - место жительства: в Москве - районы
    - девайс
  - штрафы
  - "горячность"

### Short summary
- Был собран датасет: отобраны только те кто хоть раз садился за руль
- Была выбрана целевая переменная: факт попадания в ДТП
- По пользователям из датасета были собраны данные для построения модели, следующие параметры:
   - Пол
   - Возраст
   - Стаж
   - Мобильный оператор
   - КБМ
   - Гражданство: на данный момент в базе данных такой информации нет, был создан скрипт для классификации гражданства РФ/не РФ
   - Место рождения: записано в текстовом формате. Часть данных уже была распознана. Другая часть в процессе прогона через API ДаДаты
   - Категории ВУ: наличие других категорий кроме "В"
   - Дополнительный параметр: скоринг от Qiwi
- Данные были приведены в порядок, очищены, разбиты на категории
- Модель была обучена при помощи Логистической Регрессии и Бустинга
- По результатам обучения тестовая выборка на валидации показала ROC-AUC в диапазоне от 0.69 до 0.71
- При этом на валидации было замечено очень низкое значение показателя Precision: 0.0. Это означает что модель просто предсказывает что ни один человек не попадет в ДТП.
- Был сделан вывод, что такое поведение модели обусловлено несбалансированностью выборки (в оригинальном датасете ДТПшники составляют всего 1.5% выборки, в то время как те, кто не попадал в ДТП составляют 98.5%).
- Для устранения несбалансированности было проведено овер-сэмплирование  (Over-sampling), когда соотношение классов в выборке уравнивается или приближается к более равному
- Также, для устранения bias был использован метод кросс-валидации.
- На данный момент была проведена только одна итерация по обучению искусственно сбалансированной модели и были сравнены различные методы обучения модели: Логистическая Регрессия, Бустинги, Метод Случайного Леса итд.
- Лучшие результаты показали бустинги: lightGBM и XGBoost
- На валидации они пока показывают результаты ROC-AUC ниже 0.7 (значение ROC-AUC скорее всего пострадает при калибровке Precision), однако показатель Precision поднялся до 0.09. Это означает, что из всех пользователей по которым модель предсказывает попадание в ДТП, только 9.0% действительно попадают в ДТП, т.е. происходит т.н. переобучение (over-fitting).
xgb ROC-AUC: 0.9493188716822317
xgb               precision    recall  f1-score       support
0.0            0.935797  0.831361  0.880493  37186.000000
1.0            0.849182  0.943334  0.893786  37430.000000
accuracy       0.887531  0.887531  0.887531      0.887531
macro avg      0.892490  0.887348  0.887139  74616.000000
weighted avg   0.892348  0.887531  0.887161  74616.000000

xgb ROC-AUC: 0.6769898526577152
xgb               precision    recall  f1-score       support
0.0            0.998673  0.840939  0.913043  22375.000000
1.0            0.090468  0.934037  0.164958    379.000000
accuracy       0.842489  0.842489  0.842489      0.842489
macro avg      0.544570  0.887488  0.539001  22754.000000
weighted avg   0.983546  0.842489  0.900583  22754.000000
- Чтобы уменьшить переобучение модели и несбалансированность в дальнейшем планируется протестировать обучение модели с уклоном на F1B-score с целью нахождения баланса между значениями Precision для 0 и 1.

### TO DO:
- Проверить, действительно ли модель хорошо предсказывает
- Мультиколлинеарность!
- Гибридный сэмплинг
- Дополнить фичи (место рождения)
- Проверить модель на всех юзерах без скора QIWI

### Loading the dataset

In [173]:
df = pd.read_csv('C:/Users/sgulbin/Work/Analysis/FTR_score_v2/v2/dataset.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1653596 entries, 0 to 1653595
Data columns (total 15 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   user_id                 1653596 non-null  int64  
 1   user_ext                1653596 non-null  int64  
 2   age                     1653460 non-null  float64
 3   exp                     1652029 non-null  float64
 4   sex                     1653596 non-null  object 
 5   phone_code              1653506 non-null  float64
 6   license_category        1277528 non-null  object 
 7   birth_place             1320415 non-null  object 
 8   country                 1026162 non-null  object 
 9   region                  1026164 non-null  object 
 10  city                    1026164 non-null  object 
 11  passport_citizenship    1651817 non-null  object 
 12  PassportNumber          1651817 non-null  object 
 13  PassportDepartmentCode  632036 non-null   object 
 14  Pa

### Loading additional data

In [174]:
# KBM
kbm = pd.read_csv('C:/Users/sgulbin/Work/Analysis/FTR_score_v2/v2/data_lib/kbm_19_05_2021.csv', header = None)
# Naming columns, removing non-necessary columns
kbm.columns = ['user_ext', 'kbm', 'dt']
kbm = kbm[['user_ext', 'kbm']]
# Mobile operators codes
mobile_codes = pd.read_csv('C:/Users/sgulbin/Work/Analysis/FTR_score_v2/v2/data_lib/mobile_codes_lib.csv')
# Driving license categories
license_cat = pd.read_csv('C:/Users/sgulbin/Work/Analysis/FTR_score_v2/v2/data_lib/license_cat_lib_v2.csv')
# Dropping duplicates
license_cat = license_cat.drop_duplicates(subset = 'license_category')
# Birthplaces classification. Classified using personal experience
bp_classified = pd.read_csv('C:/Users/sgulbin/Work/Analysis/FTR_score_v2/v2/data_lib/birthplaces_classified_lib.csv')
bp_region_classified = bp_classified[['bp_region', 'bp_region_group_detailed']]
bp_region_classified = bp_region_classified.drop_duplicates(subset = 'bp_region')
# Target
tg = pd.read_csv('C:/Users/sgulbin/Work/Analysis/FTR_score_v2/v2/target.csv')

In [175]:
# Qiwi
q1 = pd.read_csv('C:/Users/sgulbin/Work/Analysis/Платежеспособность/QIWI_test/deli_results_23112020.csv')
q2 = pd.read_csv('C:/Users/sgulbin/Work/Analysis/Платежеспособность/QIWI_test/v2/qiwi_exp_v2_1_results_02032021.csv')
q3 = pd.read_csv('C:/Users/sgulbin/Work/Analysis/Платежеспособность/QIWI_test/v3/data_delimobil_results_pd_fraud.csv')
q = q1.append(q2, ignore_index = True)
q = q.drop_duplicates(subset = ['ID'])
q = q[['ID', 'pd_basis_60_v3_4', 'pd_ins_kasko_v2', 'pd_ins_osago_v2_1', 'pd_ins_osago_v2_2', 'pd_basis_gm_v1']]
q1 = q1[['ID', 'pd_basis_60_v3_4', 'pd_ins_kasko_v2', 'pd_ins_osago_v2_1', 'pd_ins_osago_v2_2']]

### Enriching the dataset

In [176]:
df = pd.merge(df, kbm, on = 'user_ext', how = 'left')
df = pd.merge(df, mobile_codes, left_on = 'phone_code', right_on = 'mobile_code', how = 'left')
df = pd.merge(df, license_cat, on = 'license_category', how = 'left')
df = pd.merge(df, bp_region_classified, left_on = 'region', right_on = 'bp_region', how = 'left')
df = pd.merge(df, tg, on = 'user_id', how = 'left')

In [177]:
# Qiwi
# df = pd.merge(df, q, left_on = 'user_id', right_on = 'ID', how = 'left')
df = pd.merge(df, q1, left_on = 'user_id', right_on = 'ID', how = 'left')

In [178]:
df = df.drop(columns = ['user_ext', 'phone_code', 'license_category', 'PassportNumber', 'PassportDepartmentCode',\
                        'mobile_code', 'license_category_en', 'country', 'birth_place', 'city', 'PassportRegistration',\
                        'region', 'bp_region', 'B'])

In [179]:
# Qiwi
df = df.drop(columns = ['ID'])
# df = df.drop(columns = ['pd_basis_gm_v1'])
# df = df.drop(columns = ['pd_ins_kasko_v2', 'pd_ins_osago_v2_1', 'pd_ins_osago_v2_2'])

In [180]:
# Qiwi
df = df.loc[df.pd_basis_60_v3_4.notnull()]

In [181]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 151823 entries, 84 to 1617984
Data columns (total 18 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   user_id                   151823 non-null  int64  
 1   age                       151821 non-null  float64
 2   exp                       151815 non-null  float64
 3   sex                       151823 non-null  object 
 4   passport_citizenship      151734 non-null  object 
 5   kbm                       129581 non-null  float64
 6   mobile_operator           151819 non-null  object 
 7   A                         151823 non-null  object 
 8   C                         151823 non-null  object 
 9   D                         151823 non-null  object 
 10  E                         151823 non-null  object 
 11  Tm/Tb                     151823 non-null  object 
 12  bp_region_group_detailed  93689 non-null   object 
 13  target                    2462 non-null   

### Cleaning data

In [182]:
# Cleaning age and exp values
df['age'] = np.where(((df.age < 18)|(df.age>65)),np.nan,df.age)
df['exp'] = np.where(((df.exp < 0)|(df.exp>47)),np.nan,df.exp)
df = df.dropna(axis = 'rows', subset = ['age','exp'])
# Cleaning license category values
df = df.dropna(axis = 'rows', subset = ['A'])
# Categorizing KBM
df['kbm_grouped'] = np.where(df['kbm']<0.7,'0.5+',\
                              np.where(df['kbm']<0.8,'0.7+',\
                                       np.where(df['kbm']<0.9,'0.8+',\
                                                np.where(df['kbm']<1,'0.9+',\
                                                         np.where(df['kbm'] == 1, '1',\
                                                                  np.where(df['kbm']<2.3,'1.4+',\
                                                                           np.where(df['kbm']>=2.3,'2.3+','?')))))))
df = df.replace('?', np.NaN)
df = df.drop(columns = ['kbm'])
# Replacing True and False with 1 and 0
df = df.replace([True, False],[1, 0])
# Replacing NaN with 0 in the target column
df['target'] = df['target'].fillna(0)
# Driving license
# for col in ['A', 'C', 'D', 'E', 'Tm/Tb']:
#     df[col+'_cat'] = df[col].replace(1,col)
# df = df.assign(license = df['A_cat'].astype(str)+'-'+df['C_cat'].astype(str)+'-'+df['D_cat'].astype(str)+'-'+\
#                df['E_cat'].astype(str)+'-'+df['Tm/Tb_cat'].astype(str))
# df = df.drop(columns = ['A_cat', 'C_cat', 'D_cat', 'E_cat', 'Tm/Tb_cat'])
# df.info()

### Mean encoding

In [111]:
# LEAVING ONLY COLUMNS THAT'LL BE USED FOR THE MODEL
features = ['mobile_operator', 'sex', 'age', 'exp', 'bp_region_group_detailed', 'passport_citizenship', 'kbm_grouped', 'A',\
            'B', 'C', 'D', 'E', 'Tm/Tb']
flt = ['age', 'exp']
encoded = ['A','C', 'D', 'E', 'Tm/Tb']
to_encode = ['mobile_operator', 'sex', 'bp_region_group_detailed', 'passport_citizenship', 'kbm_grouped']
target = ['target']
df = df[features+target]

# REPLACING NA VALUES WITH 'NaN'
df = df.replace('nan', np.nan)
for feature in to_encode:
    df[feature] = df[feature].fillna('NaN')
    
# MEAN ENCODING
mean_enc_cols = []
for col in to_encode:
    means_map = df.groupby(col).target.mean()
    df[col+'_mean_enc'] = df[col].map(means_map)
    mean_enc_cols.append(col+'_mean_enc')
df = df[mean_enc_cols+flt+target]
    
# SPLITTING DATASET INTO X AND y
df = df.reset_index(drop=True)
X = df.iloc[:,:-1]
y = df.iloc[:,-1:]

### One-hot encoding categorical features

In [183]:
# LEAVING ONLY COLUMNS THAT'LL BE USED FOR THE MODEL
cols = list(df.columns)
target = ['target']
user_id = ['user_id']
features = [x for x in cols if x not in target+user_id]
cat_features = [x for x in list(df.loc[:, df.dtypes == object].columns) if x not in target+user_id]
flt_features = [x for x in list(df.loc[:, df.dtypes == float].columns) if x not in target+user_id]
encoded = ['A','C', 'D', 'E', 'Tm/Tb']
to_encode = cat_features

df = df[features+target]

# REPLACING NA VALUES WITH 'NaN'
df = df.replace('nan', np.nan)
for feature in to_encode:
    df[feature] = df[feature].fillna('NaN')
    
# SPLITTING DATASET INTO X AND y
df = df.reset_index(drop=True)
X = df[features]
y = df[target]

# ONE-HOT ENCODING
enc = preprocessing.OneHotEncoder()
X_obj = X.loc[:, X.dtypes == object]
X_enc = X[encoded]
X_flt = X.loc[:, X.dtypes == float]
enc.fit(X_obj)
X = pd.DataFrame(enc.transform(X_obj).toarray())
X = X.join(X_enc)
X = X.join(X_flt)

# CREATING VALIDATION SAMPLE
validation_smpl = X.join(y)
validation_smpl = validation_smpl.sample(frac = 0.15)
X_val = validation_smpl.iloc[:,:-1]
y_val = validation_smpl.iloc[:,-1:]
X = X.drop(X_val.index)
y = y.drop(y_val.index)

### Label encoding

In [168]:
# REMOVING ONE-HOT ENCODED COLUMNS
df = df.drop(columns = ['A', 'C', 'D', 'E', 'Tm/Tb'])

# LEAVING ONLY COLUMNS THAT'LL BE USED FOR THE MODEL
cols = list(df.columns)
target = ['target']
user_id = ['user_id']
features = [x for x in cols if x not in target+user_id]
cat_features = [x for x in list(df.loc[:, df.dtypes == object].columns) if x not in target+user_id]
flt_features = [x for x in list(df.loc[:, df.dtypes == float].columns) if x not in target+user_id]
encoded = ['A', 'C', 'D', 'E', 'Tm/Tb']
to_encode = cat_features
df = df[features+target]

# REPLACING NA VALUES WITH 'NaN'
df = df.replace('nan', np.nan)
for feature in to_encode:
    df[feature] = df[feature].fillna('NaN')

# LABEL ENCODING CATEGORICAL FEATURES
for col in cat_features:
    df[col] = df[col].astype('category')
    df[col] = df[col].cat.codes
df.info()
    
# SPLITTING DATASET INTO X AND y
df = df.reset_index(drop=True)
validation_smpl = df.sample(frac = 0.15)
df = df.drop(validation_smpl.index)
X = df[features]
y = df[target]
X_validation = validation_smpl[features]
y_validation = validation_smpl[target]

### Trying age modification

In [24]:
X['f_age'] = 1923.2*(X['age']**(-1.513))
X = X.drop(columns = ['age'])

### Trying resampling

In [112]:
smt = SMOTETomek(sampling_strategy = 'auto')
X_smt, y_smt = smt.fit_resample(X, y)

### Trying logreg out of box

In [550]:
from imblearn.under_sampling import RandomUnderSampler
# SPLITTING X AND y TO TRAIN AND TEST SAMPLES
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 7)

# # OVER-SAMPLING
# sm = SMOTE(random_state=7)
# X_res, y_res = sm.fit_resample(X_train, y_train)

# # CONVERTING X AND y DATAFRAMES TO ARRAYS
# X_train = X_res.values
# X_test = X_test.values
# y_train = y_res.values
# y_test = y_test.values

# # UNDER-SAMPLING
# undersample = RandomUnderSampler(sampling_strategy='majority')
# X_train_under, y_train_under = undersample.fit_resample(X_train, y_train)

# # CONVERTING X AND y DATAFRAMES TO ARRAYS
# X_train = X_train_under.values
# X_test = X_test.values
# y_train = y_train_under.values
# y_test = y_test.values

# CONVERTING X AND y DATAFRAMES TO ARRAYS
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

#LogisticRegression
logreg = LogisticRegression(solver='liblinear', random_state=0,C=0.01\
                           ,class_weight = 'balanced'\
                           ).fit(X_train, y_train.ravel())
y_pred = logreg.predict(X_test)

y_pred_proba = logreg.predict_proba(X_test)
y_pred_proba1 = []
for i in y_pred_proba:
    y_pred_proba1.append(i[1])

report = classification_report(y_test, y_pred, output_dict=True)
report = pd.DataFrame(report).transpose()
# report.to_csv('C:/Users/sgulbin/Work/Analysis/FTR_score_v2/v2/classification_reports/ohe_logreg_over-sample_q1.csv')
    
print(classification_report(y_test,y_pred))
print(roc_auc_score(y_test, y_pred_proba1))

              precision    recall  f1-score   support

         0.0       0.99      0.62      0.76     37313
         1.0       0.03      0.68      0.05       610

    accuracy                           0.62     37923
   macro avg       0.51      0.65      0.41     37923
weighted avg       0.98      0.62      0.75     37923

0.7045012659851771


### Trying different models with Over-Sampling and Cross-Validation out of box

In [184]:
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier

models = []
# models.append(('xgb', XGBClassifier()))
models.append(('logreg', LogisticRegression()))
# models.append(('knn', KNeighborsClassifier()))
# models.append(('dt', DecisionTreeClassifier()))
# models.append(('rf', RandomForestClassifier()))
# models.append(('lgbm', LGBMClassifier()))

seed = 7
scoring = 'roc_auc'

In [186]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import make_scorer
import warnings
warnings.filterwarnings('ignore')

oversample = RandomOverSampler(sampling_strategy='minority', random_state = seed)
smote = SMOTE(sampling_strategy=0.3, random_state = seed)
# fit and apply the transform
X_over, y_over = oversample.fit_resample(X, y)
X_sm, y_sm = smote.fit_resample(X,y)
#
undersample = RandomUnderSampler(sampling_strategy = 1, random_state = seed)
#
X_under, y_under = undersample.fit_resample(X_over, y_over)
# SPLITTING X AND y TO TRAIN AND TEST SAMPLES
X_train, X_test, y_train, y_test = train_test_split(X_over, y_over, test_size=0.25, random_state = seed)

# model = XGBClassifier()
for name, model in models:
    model.fit(X_train, y_train.values.ravel())

    # APPLYING K-FOLD CROSS VALIDATION
    kfold = RepeatedStratifiedKFold(n_splits = 5, random_state = 7, n_repeats = 3)

    # CALCULATING PREDICTION
    # y_pred_val = model.predict(X_validation)
    # y_pred_binary_val = []
    # for i in y_pred_val:
    #     if i > 0.5:
    #         y_pred_binary_val.append(1)
    #     else:
    #         y_pred_binary_val.append(0)

    # SCORING FUNCTION
    # def score(y_true, y_pred):
    #     print(classification_report(y_true, y_pred)) # print classification report
    #     return accuracy_score(y_true, y_pred) # return accuracy score

    # def score(X, y):
    #     y_pred_val = model.predict(X)
    #     y_pred_binary_val = []
    #     for i in y_pred_val:
    #         if i > 0.5:
    #             y_pred_binary_val.append(1)
    #         else:
    #             y_pred_binary_val.append(0)
    #     return recall_score(y, y_pred_binary_val)

    # VALIDATION ON OVER-SAMPLED DATASET
    cv_results = cross_val_score(model,
                                 X_train,
                                 y_train,
                                 cv = kfold,
                                 scoring = 'roc_auc')
    print(name, cv_results.mean())
    # print(classification_report(X_train, y_train))

    y_pred_val = model.predict(X_val)
    y_pred_binary_val = []
    for i in y_pred_val:
        if i > 0.5:
            y_pred_binary_val.append(1)
        else:
            y_pred_binary_val.append(0)
    print(name, classification_report(y_val, y_pred_binary_val))

    # # VALIDATION
    # cv_results_val = cross_val_score(model,
    #                                  X_validation,
    #                                  y_validation,
    #                                  cv = kfold_val,
    #                                  scoring = make_scorer(score))

logreg 0.7304065649044721
logreg               precision    recall  f1-score   support

         0.0       0.99      0.61      0.76     22366
         1.0       0.03      0.68      0.06       388

    accuracy                           0.62     22754
   macro avg       0.51      0.65      0.41     22754
weighted avg       0.97      0.62      0.75     22754



In [89]:
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import RepeatedStratifiedKFold
# from hpsklearn import HyperoptEstimator
# from hpsklearn import any_classifier
# from hpsklearn impoer any_preprocessing
import warnings
warnings.filterwarnings('ignore')
# from sklearn.model_selection import cross_val_score
oversample = RandomOverSampler(sampling_strategy='minority', random_state = seed)
# fit and apply the transform
X_over, y_over = oversample.fit_resample(X, y)
# SPLITTING X AND y TO TRAIN AND TEST SAMPLES
X_train, X_test, y_train, y_test = train_test_split(X_over, y_over, test_size=0.25, random_state = seed)
# MODEL
# model = LGBMClassifier(class_weight = 'balanced')
# CROSS-VALIDATION
for name, model in models:
    kfold = RepeatedStratifiedKFold(n_splits = 5, random_state = seed, n_repeats = 3)
#     cv_results = cross_val_score(model, X_train, y_train, cv = kfold, scoring = scoring)
#     print(name, cv_results.mean())
    # CLASSIFICATION REPORT
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred_binary = []
    for i in y_pred:
        if i > 0.5:
            y_pred_binary.append(1)
        else:
            y_pred_binary.append(0)
    report = classification_report(y_test, y_pred_binary, output_dict=True)
    report = pd.DataFrame(report).transpose()
    print(name, report)
    # VALIDATION
    cv_results_val = cross_val_score(model, X_validation, y_validation, cv = kfold, scoring = scoring)
    print(name, cv_results_val.mean())
    # CLASSIFICATION REPORT
    y_pred_val = model.predict(X_validation)
    y_pred_binary_val = []
    for i in y_pred_val:
        if i > 0.5:
            y_pred_binary_val.append(1)
        else:
            y_pred_binary_val.append(0)
    report_val = classification_report(y_validation, y_pred_binary_val, output_dict=True)
    report_val = pd.DataFrame(report_val).transpose()
    print(name, report_val)

xgb               precision    recall  f1-score       support
0.0            0.935797  0.831361  0.880493  37186.000000
1.0            0.849182  0.943334  0.893786  37430.000000
accuracy       0.887531  0.887531  0.887531      0.887531
macro avg      0.892490  0.887348  0.887139  74616.000000
weighted avg   0.892348  0.887531  0.887161  74616.000000
xgb 0.6127918786448616
xgb               precision    recall  f1-score       support
0.0            0.998769  0.833773  0.908843  22379.000000
1.0            0.086444  0.938667  0.158309    375.000000
accuracy       0.835501  0.835501  0.835501      0.835501
macro avg      0.542606  0.886220  0.533576  22754.000000
weighted avg   0.983733  0.835501  0.896474  22754.000000


### Saving True and prediction values

In [75]:
y_validation = y_validation.reset_index(drop = True)
output = y_validation.join(pd.DataFrame(y_pred_binary_val, columns = ['pred']))
output.to_csv('C:/Users/sgulbin/Work/Analysis/FTR_score_v2/v2/results/hyperopt_result1.csv', index = False)

### Tuning parameters with Hyperopt

In [139]:
from hyperopt import tpe, fmin, hp, STATUS_OK, Trials
from sklearn.metrics import fbeta_score
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import recall_score

oversample = RandomOverSampler(sampling_strategy='minority', random_state = seed)
smote = SMOTE(sampling_strategy='minority', random_state = seed)
# fit and apply the transform
X_over, y_over = oversample.fit_resample(X, y)
#
X_sm, y_sm = smote.fit_resample(X,y)
#
undersample = RandomUnderSampler(sampling_strategy = 1, random_state = seed)
#
X_under, y_under = undersample.fit_resample(X_over, y_over)
# SPLITTING X AND y TO TRAIN AND TEST SAMPLES
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.25, random_state = seed)

def objective(space):
    classifier = XGBClassifier(n_estimators = space['n_estimators'],
                            max_depth = int(space['max_depth']),
                            learning_rate = space['learning_rate'],
                            gamma = space['gamma'],
                            min_child_weight = space['min_child_weight'],
                            subsample = space['subsample'],
                            colsample_bytree = space['colsample_bytree']
                            )
    
    classifier.fit(X_train, y_train.values.ravel())

    y_pred_val = model.predict(X_validation)
    y_pred_binary_val = []
    for i in y_pred_val:
        if i > 0.5:
            y_pred_binary_val.append(1)
        else:
            y_pred_binary_val.append(0)
    recall = recall_score(y_validation, y_pred_binary_val)
#     report_val = classification_report(y_validation, y_pred_binary_val)
#     report_val = pd.DataFrame(eval(report_val)).transpose()
#     recall = report_val.iloc[1,1]

    return{'loss':1-recall,
           'status': STATUS_OK }

space = {
    'max_depth' : hp.choice('max_depth', range(5, 30, 1)),
    'learning_rate' : hp.quniform('learning_rate', 0.01, 0.5, 0.01),
    'n_estimators' : hp.choice('n_estimators', range(20, 205, 5)),
    'gamma' : hp.quniform('gamma', 0, 0.50, 0.01),
    'min_child_weight' : hp.quniform('min_child_weight', 1, 10, 1),
    'subsample' : hp.quniform('subsample', 0.1, 1, 0.01),
    'colsample_bytree' : hp.quniform('colsample_bytree', 0.1, 1.0, 0.01)
}

trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)

print("Best: ", best)

100%|████████████████████████████████████████████████| 20/20 [11:33<00:00, 34.65s/trial, best loss: 0.3866666666666667]
Best:  {'colsample_bytree': 0.71, 'gamma': 0.02, 'learning_rate': 0.02, 'max_depth': 16, 'min_child_weight': 10.0, 'n_estimators': 26, 'subsample': 0.3}


### Tuning parameters with Hyperopt for logreg

In [ ]:
from hyperopt import tpe, fmin, hp, STATUS_OK, Trials
from sklearn.metrics import fbeta_score
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import recall_score

oversample = RandomOverSampler(sampling_strategy='minority', random_state = seed)
smote = SMOTE(sampling_strategy='minority', random_state = seed)
# fit and apply the transform
X_over, y_over = oversample.fit_resample(X, y)
#
X_sm, y_sm = smote.fit_resample(X,y)
#
undersample = RandomUnderSampler(sampling_strategy = 1, random_state = seed)
#
X_under, y_under = undersample.fit_resample(X_over, y_over)
# SPLITTING X AND y TO TRAIN AND TEST SAMPLES
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.25, random_state = seed)

def objective(space):
    classifier = XGBClassifier(classifier_penalty = space['classifier_penalty'],
                            max_depth = int(space['max_depth'])
                            )
    
    classifier.fit(X_train, y_train.values.ravel())

    y_pred_val = model.predict(X_val)
    y_pred_binary_val = []
    for i in y_pred_val:
        if i > 0.5:
            y_pred_binary_val.append(1)
        else:
            y_pred_binary_val.append(0)
    recall = recall_score(y_val, y_pred_binary_val)
#     report_val = classification_report(y_validation, y_pred_binary_val)
#     report_val = pd.DataFrame(eval(report_val)).transpose()
#     recall = report_val.iloc[1,1]

    return{'loss':1-recall,
           'status': STATUS_OK }

space = {
    'classifier__penalty' : ['l1', 'l2'],
    'classifier__C' : np.logspace(-4, 4, 20)
}

trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)

print("Best: ", best)

In [229]:
# [3]*(int(11/3))
# 11%3
a=[1,99,0]
b = sorted(a)
b
# b
# print(a.sort())

[0, 1, 99]

### Trying lightGBM out of box

In [315]:
# SPLITTING X AND y TO TRAIN AND TEST SAMPLES
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 7)

# OVER-SAMPLING
sm = SMOTE(random_state=7)
X_res, y_res = sm.fit_resample(X_train, y_train)

# CONVERTING X AND y DATAFRAMES TO ARRAYS
X_train = X_res.values
X_test = X_test.values
y_train = y_res.values
y_test = y_test.values

# CONVERTING X AND y DATAFRAMES TO ARRAYS
# X_train = X_train.values
# X_test = X_test.values
# y_train = y_train.values
# y_test = y_test.values

# LIGHTGBM
d_train = lgb.Dataset(X_train, label=y_train)
params = {}
params['learning_rate'] = 0.003
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'binary_logloss'
params['sub_feature'] = 0.5
params['num_leaves'] = 10
params['min_data'] = 50
params['max_depth'] = 10
clf = lgb.train(params, d_train, 100)

y_pred=clf.predict(X_test)

y_pred_binary = []
for i in y_pred:
    if i > 0.5:
        y_pred_binary.append(1)
    else:
        y_pred_binary.append(0)
        
report = classification_report(y_test, y_pred_binary, output_dict=True)
report = pd.DataFrame(report).transpose()
report.to_csv('C:/Users/sgulbin/Work/Analysis/FTR_score_v2/v2/classification_reports/le_lgbm_over-sample_q1.csv')

print('ROC-AUC:'+roc_auc_score(y_test, y_pred).astype(str))

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:154: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')


[LightGBM] [Info] Number of positive: 111918, number of negative: 111918
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008104 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1669
[LightGBM] [Info] Number of data points in the train set: 223836, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
ROC-AUC:0.6976940968580809


### Checking feature importance

In [188]:
# LOGREG
coefs = pd.DataFrame(model.coef_)
coefs.columns = list(enc.get_feature_names())+encoded+flt_features

coefs = coefs.T
print(coefs.to_string())

# lightGBM
# lgb.plot_importance(clf)

#XGBoost
# model.get_booster().get_score(importance_type="gain")

                                             0
x0_Ж                                  0.000834
x0_М                                  0.119244
x1_NaN                                0.003744
x1_Russian                           -0.203264
x1_foreign                            0.319598
x2_NaN                               -0.004777
x2_Билайн                            -0.032562
x2_Др.                                0.053010
x2_МТС                               -0.050627
x2_Мегафон                           -0.018773
x2_Теле2                              0.173807
x3_NaN                                0.086587
x3_Азербайджан                       -0.008440
x3_Алтайский край                     0.437855
x3_Амурская область                   0.049029
x3_Армения                           -0.021281
x3_Архангельская область              0.020674
x3_Астраханская область               0.029396
x3_Беларусь                           0.022105
x3_Белгородская область              -0.071112
x3_Брянская о